In [ ]:
#Import necessary Libraries
from flask import Flask, render_template, request, redirect, url_for
import sqlalchemy
import flask_sqlalchemy
import sqlite3
import os
from threading import Thread
from IPython.display import display, Javascript


#Initialise the Flask application
app = Flask(__name__)

#Store Registered users in memory (for simplicity)
users = []


#Function to connect to the SQLite database and initialise the table
def init_db():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL,
        password TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
    conn.commit()   
    conn.close()

#Route for the Home Page
@app.route('/')
def home():
    return render_template('home.html')

@app.route('/logged_in')
def logged_in():
    return render_template('logged_in.html')

#Route for the About page
@app.route('/about')
def about():
    return render_template('about.html')

#Route for the Contact Us Page
@app.route('/contact')
def contact():
    return render_template('contact.html')

#Route to display registered users
@app.route('/users')
def users():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT username, created_at FROM users")
    users = cursor.fetchall()
    conn.close()
    
    return render_template('users.html', users=users)

#Route for the Registration Page
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        
        #Insert the new user into the database
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute("Insert INTO users (username, password) VALUES (?,?)", (username,password))
        conn.commit()
        conn.close()
    
        return redirect('/users')
    return render_template('register.html')

#Route for the Login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
    
        #try to match the login against the passed in values
        conn = sqlite3.connect('users.db')
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT() FROM users where username='?' and password='?'", (username,password))
        matches = cursor.fetchone()[0]
        conn.close()
    
        if matches == 1:
            return redirect(url_for('/logged_in'))
    
    
    return render_template('login.html')
        
def run_flask():
    #Display a link to open the Flask app in a new browser tab
    #display(Javascript('window.open("/proxy/5000/","_blank")'))
    app.run(host='0.0.0.0', port=5000)
    
# Start Flask app in a thread
Thread(target=run_flask).start()

#start the flask application
init_db()
run_flask()


 * Serving Flask app '__main__'
 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.0.0.105:5000
Press CTRL+C to quit
